In [1]:
#| defualt_exp rf_model

In [2]:
#| export


from fastai.tabular.all import *

from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance


import xgboost as xgb

import seaborn as sns

import optuna

import json

In [3]:
#| export
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [4]:
#| export
comp = 'playground-series-s3e11'
path = setup_comp(comp, install='fastai')

In [5]:
#| export
df_train = pd.read_csv(path/'train.csv', low_memory=False)
df_test = pd.read_csv(path/'test.csv', low_memory=False)
df_comb = pd.concat([df_train, df_test], ignore_index=True)

In [6]:
#| export
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)
df_comb.drop(['id'], axis=1, inplace=True)

In [7]:
#| export
df_train['store_sales_per_children'] = df_train['store_sales(in millions)'] / df_train['total_children']

In [8]:
#| export
for column in df_train.columns:
    if (list(df_train[column].unique()) == [0.0, 1.0]):
        df_train.loc[:, column] = df_train[column].astype('bool')

In [9]:
#| export
train_idxs = np.arange(len(df_train))
test_idxs = np.arange(len(df_train), len(df_comb))

In [10]:
#| export
dep_var = 'cost'
procs = [Categorify, FillMissing, Normalize]
cont, cat = cont_cat_split(df_comb, max_card=1, dep_var=dep_var)
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

In [11]:
#| export
df_train = df_comb.iloc[train_idxs]
df_test = df_comb.iloc[test_idxs]

In [13]:
#| export
to_final = TabularPandas(df_train, procs, cat, cont, y_names=dep_var, splits=splits)
test_final = TabularPandas(df_test, procs, cat, cont, y_names=None, splits=None)
dls_final = to_final.dataloaders(bs=1024)

### Random Forest Model

In [15]:
#| export
xs, y = to_final.train.xs, to_final.train.y
valid_xs, valid_y = to_final.valid.xs, to_final.valid.y
test_xs = test_final.train.xs

In [16]:
#| export
def rf(xs, y, n_estimators=40, max_samples=200_000, max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators, 
                                 max_samples=max_samples, max_features=max_features,
                                 min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

#### RF Optuna Optimization

In [17]:
rf_study = False

In [18]:
def rf_param(trial, xs, y):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_samples = trial.suggest_int('max_samples', 100, 200_000)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = rf(xs, y, n_estimators=n_estimators, max_samples=max_samples, 
               max_features=max_features, min_samples_leaf=min_samples_leaf)
    
    score = cross_val_score(model, xs, y, cv=5, scoring='neg_mean_squared_error')
    return math.sqrt(-1 * score.mean())

In [19]:
if rf_study == True:
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: rf_param(trial, xs, y), n_trials=50)
    with open('./training_params/rf_params.json', 'w') as fp:
        json.dump(study.best_params, fp) 

In [20]:
with open('./training_params/rf_params.json', 'r') as fp:
    rf_best_params = json.load(fp)

In [21]:
#| export
def r_mse(pred, y):
    return round(math.sqrt(((pred-y)**2).mean()), 6)

In [22]:
#| export
def m_rmse(m, xs, y):
    return r_mse(m.predict(xs), y)

In [23]:
#| export
m = rf(xs, y, **rf_best_params)

In [24]:
rf_mse = m_rmse(m, valid_xs, valid_y)
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(26.493274, 28.126384)

In [25]:
#| export
rf_preds = m.predict(test_xs)
np.savetxt('./predictions/rf_preds.csv', rf_preds, delimiter=',')

In [26]:
rf_preds = np.loadtxt('./predictions/rf_preds.csv', delimiter=',')

#### RF Feature Importance

In [27]:
#| export
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp': m.feature_importances_}).sort_values('imp', ascending=False) 

In [28]:
fi = rf_feat_importance(m, xs)
fi

,cols,imp
9,store_sqft,0.182804
0,store_sales(in millions),0.164107
5,gross_weight,0.149167
2,total_children,0.108639
8,units_per_case,0.104862
4,avg_cars_at home(approx).1,0.087429
3,num_children_at_home,0.071035
14,florist,0.041245
1,unit_sales(in millions),0.021650
6,recyclable_package,0.019942


In [29]:
rf_preds

array([100.80295445,  95.06213634,  89.34147955, ..., 102.47697174,
       113.67668586, 116.88759135])

In [31]:
#| export
np.savetxt('./predictions/rf_preds.csv', rf_preds, delimiter=',')

In [32]:
#| export
rf_preds = np.loadtxt('./predictions/rf_preds.csv', delimiter=',')

In [33]:
Path('./rf_submissions').mkdir(exist_ok=True, parents=True)
sample_df = pd.read_csv(path/'sample_submission.csv')
sample_df['cost'] = rf_preds
sample_df.to_csv('./rf_submissions/submission.csv', index=False)

In [34]:
new_sub_nums = sample_df['cost'].to_numpy()
benchmark_df = pd.read_csv('benchmark_submission.csv')
benchmark_nums = benchmark_df['cost'].to_numpy()
diffs = np.mean(np.abs(new_sub_nums - benchmark_nums))

In [35]:
diffs

4.973435514391007

In [36]:
submit = True

In [37]:
if not iskaggle and submit:
    from kaggle import api
    api.competition_submit_cli(file_name='./rf_submissions/submission.csv', message="rf_model", competition=comp)

100%|██████████| 5.78M/5.78M [00:02<00:00, 2.40MB/s]


In [43]:
submission_history = api.competitions_submissions_list(id=comp)

In [44]:
nn_submission_history = [x for x in submission_history if x['descriptionNullable'] == 'rf_model']

In [45]:
most_recent_rf_submission = max(nn_submission_history, key=lambda x: x['date'])

In [46]:
most_recent_rf_submission

{'totalBytesNullable': 6060976,
 'descriptionNullable': 'rf_model',
 'errorDescriptionNullable': None,
 'fileNameNullable': 'submission.csv',
 'publicScoreNullable': '0.29889',
 'privateScoreNullable': None,
 'submittedByNullable': 'Peter Bull',
 'submittedByRefNullable': 'peterbull',
 'teamNameNullable': 'Peter Bull',
 'urlNullable': 'https://www.kaggle.com/submissions/30956274/30956274.raw',
 'ref': 30956274,
 'totalBytes': 6060976,
 'hasTotalBytes': True,
 'date': '2023-04-01T11:07:56.833Z',
 'description': 'rf_model',
 'hasDescription': True,
 'errorDescription': '',
 'hasErrorDescription': False,
 'fileName': 'submission.csv',
 'hasFileName': True,
 'publicScore': '0.29889',
 'hasPublicScore': True,
 'privateScore': '',
 'hasPrivateScore': False,
 'status': 'complete',
 'submittedBy': 'Peter Bull',
 'hasSubmittedBy': True,
 'submittedByRef': 'peterbull',
 'hasSubmittedByRef': True,
 'teamName': 'Peter Bull',
 'hasTeamName': True,
 'url': 'https://www.kaggle.com/submissions/3095627

In [ ]:
import nbdev
nbdev.export.nb_export('nn_model.ipynb', 'nn_model')
print("export successful")